Getting UseCase 3

In [ ]:
cd /home/karen/Documents/GitHub/Muscle_wasting

In [ ]:
import mirna_scoring.mirna_impact as mis
import mirna_scoring.mirna_influence_plots as mi_plot
from network.network_processing import load_graph

In [ ]:
network = load_graph(f"network/Networks_pkl/complete_n_tf_mirnas__UseCase1_cutoff_0.95.pkl")
session_name = 'UseCase_3'

In [ ]:
mirna_network=mis.mirna_network(network)
evaluation = mis.mirna_evaluation(mirna_network, session_name)

In [ ]:
len(evaluation.get_all_mirnas())

In [ ]:
evaluation_df = evaluation.score(steps=10, sample_size=10, dds_threshold=2, pathway_keywords=['ATP', 'MUSCLE', "MYO"])

In [ ]:
aa=evaluation.select_mirnas()
dict(zip(list(aa),[list(aa).count(i) for i in list(aa)]))

In [ ]:
n_clusters = len(mirna_network.miR_nodes)//3
evaluation.cluster_mirnas(n_clusters=n_clusters)

In [ ]:
dist_df = mirna_network.get_mirnas_similarity()
mi_plot.plot_mirnas_similarirty(dist_df)

In [ ]:
evaluation.create_report_clusters()

In [ ]:
genes = mirna_network.network.nodes
genes = list(set(genes))

In [ ]:
len(genes)

In [ ]:
import decoupler as dc
from mirkitten.plot_GSEA_ORA import plot_ora_results
enriched = dc.get_ora_df(
            df=genes,
            net=evaluation.msigdb,
            source='geneset',
            target='genesymbol'
        )
pathway_df = enriched[enriched['FDR p-value'] < 0.1]
pathway_df.index = pathway_df["Term"]
#pathway_df.set_index("Term", inplace=True)  # Set "Term" as index
enriched_pathways = pathway_df['Combined score']
plot_ora_results(pathway_df, top_n=10, figsize=(12, 8), scale_odds_ratio=20, 
                     fontsize_title=16, fontsize_subtitle=14, fontsize_text=12,)

In [ ]:
mirnas_selected = list(set(aa))
mirna_clusters = evaluation.clustered_mirnas

In [ ]:
import umap
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
selected_dist_df = dist_df.loc[mirnas_selected, mirnas_selected]
# Compute UMAP embedding
reducer = umap.UMAP(metric="precomputed", random_state=42)
embedding = reducer.fit_transform(selected_dist_df)

# Create a dataframe with UMAP results and cluster labels

plot_df = pd.DataFrame(embedding, columns=['UMAP1', 'UMAP2'], index=selected_dist_df.index)
plot_df['Cluster'] = mirna_clusters['Cluster']

# Plot the clusters
plt.figure(figsize=(10, 7))
sns.scatterplot(x='UMAP1', y='UMAP2', hue='Cluster', data=plot_df, palette='tab10', alpha=0.7)

# Add labels for each miRNA
for i, txt in enumerate(plot_df.index):
    plt.annotate(txt, (plot_df['UMAP1'][i], plot_df['UMAP2'][i]), fontsize=8, alpha=0.7)

plt.title("UMAP Visualization of Spectral Clustering")
plt.legend(title="Cluster", bbox_to_anchor=(1, 1))
plt.show()


In [ ]:
mi_plot.plot_mirnas_similarirty(selected_dist_df)